# 0. Libraries and importing the data

In [2]:
import pandas as pd
import numpy as np
import os

We import the training,test data and their combination after the modifications in the EDA:

In [3]:
# We import the training dataset
csv_path1 = os.path.join("preprocessed_datasets", "03_train_dataset_v2.csv")
df_train = pd.read_csv(csv_path1)

csv_path2 = os.path.join("preprocessed_datasets", "04_test_dataset_v2.csv")
df_test = pd.read_csv(csv_path2)

csv_path3 = os.path.join("preprocessed_datasets", "05_concat_dataset_v2.csv")
df = pd.read_csv(csv_path3)

In [4]:
df.head()

,HOSPITAL_EXPIRE_FLAG,subject_id,hadm_id,icustay_id,HeartRate_Min,HeartRate_Max,HeartRate_Mean,SysBP_Min,SysBP_Max,SysBP_Mean,...,RELIGION,MARITAL_STATUS,ETHNICITY,FIRST_CAREUNIT,train,cci_index,icustays_per_hospstay,icustays_per_subject,number_comorbidities,age_admission
0,0.0,77502,151200,299699,89.0,116.0,102.677419,97.0,150.0,126.000000,...,PROTESTANT QUAKER,DIVORCED,BLACK/AFRICAN AMERICAN,MICU,1,2,1,2,28,63
1,0.0,44346,140114,250021,74.0,114.0,92.204082,87.0,160.0,122.000000,...,EPISCOPALIAN,DIVORCED,WHITE,TSICU,1,0,1,1,9,62
2,0.0,92438,118589,288511,59.0,89.0,70.581395,88.0,160.0,120.933333,...,CATHOLIC,MARRIED,WHITE,CSRU,1,0,1,2,10,73
3,1.0,83663,125553,278204,75.0,86.0,80.400000,74.0,102.0,85.227273,...,CATHOLIC,MARRIED,BLACK/AFRICAN AMERICAN,MICU,1,0,1,1,11,62
4,0.0,85941,181409,292581,77.0,107.0,91.020408,95.0,150.0,108.625000,...,NOT SPECIFIED,DIVORCED,WHITE,MICU,1,0,1,1,24,62


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20885 entries, 0 to 20884
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   HOSPITAL_EXPIRE_FLAG   8000 non-null   float64
 1   subject_id             20885 non-null  int64  
 2   hadm_id                20885 non-null  int64  
 3   icustay_id             20885 non-null  int64  
 4   HeartRate_Min          18698 non-null  float64
 5   HeartRate_Max          18698 non-null  float64
 6   HeartRate_Mean         18698 non-null  float64
 7   SysBP_Min              18677 non-null  float64
 8   SysBP_Max              18677 non-null  float64
 9   SysBP_Mean             18677 non-null  float64
 10  DiasBP_Min             18676 non-null  float64
 11  DiasBP_Max             18676 non-null  float64
 12  DiasBP_Mean            18676 non-null  float64
 13  MeanBP_Min             18699 non-null  float64
 14  MeanBP_Max             18699 non-null  float64
 15  Me

Remaining features:

In [6]:
# Numerical features

all_numerical_features= df.select_dtypes(include=['number']).columns.tolist()

not_numerical = ['HOSPITAL_EXPIRE_FLAG', 'subject_id', 'hadm_id', 'icustay_id', 'train'] 

numerical_features = [col for col in all_numerical_features if col not in not_numerical]

# Categorical features

categorical_features = df.select_dtypes(include=['object']).columns.tolist()

print(f'Number of numerical features: {len(numerical_features)}\n', numerical_features, '\n')

print(f'Number of categorical features: {len(categorical_features)}\n', categorical_features, '\n')

print(f'Total number of features: {len(numerical_features) + len(categorical_features)}')

Number of numerical features: 29
 ['HeartRate_Min', 'HeartRate_Max', 'HeartRate_Mean', 'SysBP_Min', 'SysBP_Max', 'SysBP_Mean', 'DiasBP_Min', 'DiasBP_Max', 'DiasBP_Mean', 'MeanBP_Min', 'MeanBP_Max', 'MeanBP_Mean', 'RespRate_Min', 'RespRate_Max', 'RespRate_Mean', 'TempC_Min', 'TempC_Max', 'TempC_Mean', 'SpO2_Min', 'SpO2_Max', 'SpO2_Mean', 'Glucose_Min', 'Glucose_Max', 'Glucose_Mean', 'cci_index', 'icustays_per_hospstay', 'icustays_per_subject', 'number_comorbidities', 'age_admission'] 

Number of categorical features: 7
 ['GENDER', 'ADMISSION_TYPE', 'INSURANCE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY', 'FIRST_CAREUNIT'] 

Total number of features: 36


# 3. Preprocessing

## 3.1. Feature removal
For now, no variable will be removed from the dataset.

## 3.2. Dealing with categorical variables
In this section, we apply:
- Simple imputation (of the mode) to the only categorical variable with missing values: `MARITAL_STATUS`. We impute separately for the training and test datasets, to avoid data leakage.
- Check whether the categories for all categorical features are the same in the training and test datasets.
- OHE to those categorical features relevant for prediction which have the same categories in both training and test datasets. 

The same transformation can be applied to the test set without having data leakage.

### Separate imputation of `MARITAL_STATUS` for training and test datasets

In [7]:
df_train2 = df_train.copy()
df_test2 = df_test.copy()

In [8]:
# Impute with mode for training dataset
df_train2['MARITAL_STATUS'] = df_train2['MARITAL_STATUS'].fillna(df_train2['MARITAL_STATUS'].mode()[0])  # Select 1st mode

# Impute with mode for test dataset
df_test2['MARITAL_STATUS'] = df_test2['MARITAL_STATUS'].fillna(df_test2['MARITAL_STATUS'].mode()[0])  # Select 1st mode

In [9]:
df_train2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   HOSPITAL_EXPIRE_FLAG   8000 non-null   float64
 1   subject_id             8000 non-null   int64  
 2   hadm_id                8000 non-null   int64  
 3   icustay_id             8000 non-null   int64  
 4   HeartRate_Min          7167 non-null   float64
 5   HeartRate_Max          7167 non-null   float64
 6   HeartRate_Mean         7167 non-null   float64
 7   SysBP_Min              7160 non-null   float64
 8   SysBP_Max              7160 non-null   float64
 9   SysBP_Mean             7160 non-null   float64
 10  DiasBP_Min             7160 non-null   float64
 11  DiasBP_Max             7160 non-null   float64
 12  DiasBP_Mean            7160 non-null   float64
 13  MeanBP_Min             7167 non-null   float64
 14  MeanBP_Max             7167 non-null   float64
 15  Mean

### Checking consistency of categories across training and test datasets

If the categories are the same in the training and test datasets, we can already apply OHE at this stage.

In [10]:
differences = {}  # Initialize dictionary to store different categories

for col in categorical_features:
    unique_train = set(df_train2[col].dropna().unique())  # Get unique categories in training
    unique_test = set(df_test2[col].dropna().unique())  # Get unique categories in test
    
    if unique_train != unique_test:
        differences[col] = {
            'only_in_training': unique_train - unique_test,
            'only_in_test': unique_test - unique_train
        }

print(differences)

{'RELIGION': {'only_in_training': {'HEBREW'}, 'only_in_test': set()}, 'ETHNICITY': {'only_in_training': set(), 'only_in_test': {'ASIAN - OTHER'}}}


There are some categories only in the training or test datasets. For that reason, and since:
- There are several categories for both the `RElIGION` and `ETHNICITY` variables, which will make data very sparse after applying OHE, and
- Neither of them is expected to have a high predictive power (compared to other, more relevant variables).

In [11]:
df_concat2 = pd.concat([df_train2, df_test2])

df_concat2.drop(columns = ['RELIGION', 'ETHNICITY'], inplace = True)

In [12]:
df_concat2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20885 entries, 0 to 12884
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   HOSPITAL_EXPIRE_FLAG   8000 non-null   float64
 1   subject_id             20885 non-null  int64  
 2   hadm_id                20885 non-null  int64  
 3   icustay_id             20885 non-null  int64  
 4   HeartRate_Min          18698 non-null  float64
 5   HeartRate_Max          18698 non-null  float64
 6   HeartRate_Mean         18698 non-null  float64
 7   SysBP_Min              18677 non-null  float64
 8   SysBP_Max              18677 non-null  float64
 9   SysBP_Mean             18677 non-null  float64
 10  DiasBP_Min             18676 non-null  float64
 11  DiasBP_Max             18676 non-null  float64
 12  DiasBP_Mean            18676 non-null  float64
 13  MeanBP_Min             18699 non-null  float64
 14  MeanBP_Max             18699 non-null  float64
 15  MeanBP_

In [13]:
df_concat2.head()

,HOSPITAL_EXPIRE_FLAG,subject_id,hadm_id,icustay_id,HeartRate_Min,HeartRate_Max,HeartRate_Mean,SysBP_Min,SysBP_Max,SysBP_Mean,...,ADMISSION_TYPE,INSURANCE,MARITAL_STATUS,FIRST_CAREUNIT,train,cci_index,icustays_per_hospstay,icustays_per_subject,number_comorbidities,age_admission
0,0.0,77502,151200,299699,89.0,116.0,102.677419,97.0,150.0,126.000000,...,EMERGENCY,Medicare,DIVORCED,MICU,1,2,1,2,28,63
1,0.0,44346,140114,250021,74.0,114.0,92.204082,87.0,160.0,122.000000,...,EMERGENCY,Private,DIVORCED,TSICU,1,0,1,1,9,62
2,0.0,92438,118589,288511,59.0,89.0,70.581395,88.0,160.0,120.933333,...,EMERGENCY,Medicare,MARRIED,CSRU,1,0,1,2,10,73
3,1.0,83663,125553,278204,75.0,86.0,80.400000,74.0,102.0,85.227273,...,EMERGENCY,Private,MARRIED,MICU,1,0,1,1,11,62
4,0.0,85941,181409,292581,77.0,107.0,91.020408,95.0,150.0,108.625000,...,EMERGENCY,Medicaid,DIVORCED,MICU,1,0,1,1,24,62


In [14]:
# Update features list

# Numerical features

all_numerical_features= df_concat2.select_dtypes(include=['number']).columns.tolist()

not_numerical = ['HOSPITAL_EXPIRE_FLAG', 'subject_id', 'hadm_id', 'icustay_id', 'train'] 

numerical_features = [col for col in all_numerical_features if col not in not_numerical]

# Categorical features

categorical_features = df_concat2.select_dtypes(include=['object']).columns.tolist()

print(f'Number of numerical features: {len(numerical_features)}\n', numerical_features, '\n')

print(f'Number of categorical features: {len(categorical_features)}\n', categorical_features, '\n')

print(f'Total number of features: {len(numerical_features) + len(categorical_features)}')

Number of numerical features: 29
 ['HeartRate_Min', 'HeartRate_Max', 'HeartRate_Mean', 'SysBP_Min', 'SysBP_Max', 'SysBP_Mean', 'DiasBP_Min', 'DiasBP_Max', 'DiasBP_Mean', 'MeanBP_Min', 'MeanBP_Max', 'MeanBP_Mean', 'RespRate_Min', 'RespRate_Max', 'RespRate_Mean', 'TempC_Min', 'TempC_Max', 'TempC_Mean', 'SpO2_Min', 'SpO2_Max', 'SpO2_Mean', 'Glucose_Min', 'Glucose_Max', 'Glucose_Mean', 'cci_index', 'icustays_per_hospstay', 'icustays_per_subject', 'number_comorbidities', 'age_admission'] 

Number of categorical features: 5
 ['GENDER', 'ADMISSION_TYPE', 'INSURANCE', 'MARITAL_STATUS', 'FIRST_CAREUNIT'] 

Total number of features: 34


### OHE
Since there are no more missing values in the remaining categorical variables, we can apply OHE directly to the training and test sets.

In [15]:
def ohe_join(dataframe: pd.DataFrame, variables_to_encode: list, na_column: bool, drop_first: bool):
    for column in variables_to_encode:
        # We encode OHE each variable, creating an additional column which stores NaN
        ohe = pd.get_dummies(data = dataframe[column], drop_first = drop_first, dummy_na = na_column)
        
        # Rename columns to avoid potential conflicts
        ohe.columns = [f"{column}_{str(col)}" for col in ohe.columns]
        
        # Concatenate along the columns the encoded dataframe to the input dataframe
        dataframe = pd.concat([dataframe, ohe], axis=1)
        
        # Drop the original columns from the dataframe
        dataframe = dataframe.drop(labels=column, axis=1)
    
    return dataframe

In [16]:
# Create copy of the data frame
df_concat3 = df_concat2.copy()

# We apply OHE to the combined dataset
df_concat3 = ohe_join(
    dataframe = df_concat3, 
    variables_to_encode = categorical_features, 
    na_column = False,
    drop_first = False)

In [17]:
df_concat3.head()

,HOSPITAL_EXPIRE_FLAG,subject_id,hadm_id,icustay_id,HeartRate_Min,HeartRate_Max,HeartRate_Mean,SysBP_Min,SysBP_Max,SysBP_Mean,...,MARITAL_STATUS_MARRIED,MARITAL_STATUS_SEPARATED,MARITAL_STATUS_SINGLE,MARITAL_STATUS_UNKNOWN (DEFAULT),MARITAL_STATUS_WIDOWED,FIRST_CAREUNIT_CCU,FIRST_CAREUNIT_CSRU,FIRST_CAREUNIT_MICU,FIRST_CAREUNIT_SICU,FIRST_CAREUNIT_TSICU
0,0.0,77502,151200,299699,89.0,116.0,102.677419,97.0,150.0,126.000000,...,False,False,False,False,False,False,False,True,False,False
1,0.0,44346,140114,250021,74.0,114.0,92.204082,87.0,160.0,122.000000,...,False,False,False,False,False,False,False,False,False,True
2,0.0,92438,118589,288511,59.0,89.0,70.581395,88.0,160.0,120.933333,...,True,False,False,False,False,False,True,False,False,False
3,1.0,83663,125553,278204,75.0,86.0,80.400000,74.0,102.0,85.227273,...,True,False,False,False,False,False,False,True,False,False
4,0.0,85941,181409,292581,77.0,107.0,91.020408,95.0,150.0,108.625000,...,False,False,False,False,False,False,False,True,False,False


## 3.3. Cleaning the data

### 3.3.1. Handling missing values
First of all, after the preprocessing we have applied we should check, again, which variables still have missing values in order to decide how to deal with them.

In [18]:
# Count of null values per column
df_na = df_concat3.isnull().sum()

# Filter only columns with missing values
missing_values = df_na[df_na > 0]

# Compute percentage of missing values
missing_values_percentage = (missing_values / len(df_concat3)) * 100

# Display the result
print("Columns with missing values:")
print(missing_values)
print("\nPercentage of missing values:")
print(missing_values_percentage)

Columns with missing values:
HOSPITAL_EXPIRE_FLAG    12885
HeartRate_Min            2187
HeartRate_Max            2187
HeartRate_Mean           2187
SysBP_Min                2208
SysBP_Max                2208
SysBP_Mean               2208
DiasBP_Min               2209
DiasBP_Max               2209
DiasBP_Mean              2209
MeanBP_Min               2186
MeanBP_Max               2186
MeanBP_Mean              2186
RespRate_Min             2189
RespRate_Max             2189
RespRate_Mean            2189
TempC_Min                2497
TempC_Max                2497
TempC_Mean               2497
SpO2_Min                 2203
SpO2_Max                 2203
SpO2_Mean                2203
Glucose_Min               253
Glucose_Max               253
Glucose_Mean              253
dtype: int64

Percentage of missing values:
HOSPITAL_EXPIRE_FLAG    61.694996
HeartRate_Min           10.471630
HeartRate_Max           10.471630
HeartRate_Mean          10.471630
SysBP_Min               10.572181
SysBP_M

**Relevant insights**:
- There is no single column with an excessively high number of null values (e.g., over 50%).
- Only the variables of patient vitals have missing values, in general by a bit over 10%. That is not very problematic. In the training pipeline, I will try different imputation approaches to fill null values consistently (mainly, simple imputation and KNN imputation).

### 3.3.2. Handling outliers

If outliers are handled, they will be handled after the training-validation split.

## 3.4. Saving dataframes
In this step, we export the dataframes to csv so they can be easily imported for other steps in the pipeline. 

In [19]:
df_concat3.head()

,HOSPITAL_EXPIRE_FLAG,subject_id,hadm_id,icustay_id,HeartRate_Min,HeartRate_Max,HeartRate_Mean,SysBP_Min,SysBP_Max,SysBP_Mean,...,MARITAL_STATUS_MARRIED,MARITAL_STATUS_SEPARATED,MARITAL_STATUS_SINGLE,MARITAL_STATUS_UNKNOWN (DEFAULT),MARITAL_STATUS_WIDOWED,FIRST_CAREUNIT_CCU,FIRST_CAREUNIT_CSRU,FIRST_CAREUNIT_MICU,FIRST_CAREUNIT_SICU,FIRST_CAREUNIT_TSICU
0,0.0,77502,151200,299699,89.0,116.0,102.677419,97.0,150.0,126.000000,...,False,False,False,False,False,False,False,True,False,False
1,0.0,44346,140114,250021,74.0,114.0,92.204082,87.0,160.0,122.000000,...,False,False,False,False,False,False,False,False,False,True
2,0.0,92438,118589,288511,59.0,89.0,70.581395,88.0,160.0,120.933333,...,True,False,False,False,False,False,True,False,False,False
3,1.0,83663,125553,278204,75.0,86.0,80.400000,74.0,102.0,85.227273,...,True,False,False,False,False,False,False,True,False,False
4,0.0,85941,181409,292581,77.0,107.0,91.020408,95.0,150.0,108.625000,...,False,False,False,False,False,False,False,True,False,False


In [20]:
# Again, we divide the concatenated data frame into training and test

# Divide again into test and training datasets
df_train3 = df_concat3[df_concat3['train'] == 1].copy()
df_test3 = df_concat3[df_concat3['train'] == 0].copy()

# Drop target from test
df_test3.drop(columns = ['HOSPITAL_EXPIRE_FLAG'], inplace = True)

In [21]:
df_train3.head()

,HOSPITAL_EXPIRE_FLAG,subject_id,hadm_id,icustay_id,HeartRate_Min,HeartRate_Max,HeartRate_Mean,SysBP_Min,SysBP_Max,SysBP_Mean,...,MARITAL_STATUS_MARRIED,MARITAL_STATUS_SEPARATED,MARITAL_STATUS_SINGLE,MARITAL_STATUS_UNKNOWN (DEFAULT),MARITAL_STATUS_WIDOWED,FIRST_CAREUNIT_CCU,FIRST_CAREUNIT_CSRU,FIRST_CAREUNIT_MICU,FIRST_CAREUNIT_SICU,FIRST_CAREUNIT_TSICU
0,0.0,77502,151200,299699,89.0,116.0,102.677419,97.0,150.0,126.000000,...,False,False,False,False,False,False,False,True,False,False
1,0.0,44346,140114,250021,74.0,114.0,92.204082,87.0,160.0,122.000000,...,False,False,False,False,False,False,False,False,False,True
2,0.0,92438,118589,288511,59.0,89.0,70.581395,88.0,160.0,120.933333,...,True,False,False,False,False,False,True,False,False,False
3,1.0,83663,125553,278204,75.0,86.0,80.400000,74.0,102.0,85.227273,...,True,False,False,False,False,False,False,True,False,False
4,0.0,85941,181409,292581,77.0,107.0,91.020408,95.0,150.0,108.625000,...,False,False,False,False,False,False,False,True,False,False


In [22]:
df_test3.head()

,subject_id,hadm_id,icustay_id,HeartRate_Min,HeartRate_Max,HeartRate_Mean,SysBP_Min,SysBP_Max,SysBP_Mean,DiasBP_Min,...,MARITAL_STATUS_MARRIED,MARITAL_STATUS_SEPARATED,MARITAL_STATUS_SINGLE,MARITAL_STATUS_UNKNOWN (DEFAULT),MARITAL_STATUS_WIDOWED,FIRST_CAREUNIT_CCU,FIRST_CAREUNIT_CSRU,FIRST_CAREUNIT_MICU,FIRST_CAREUNIT_SICU,FIRST_CAREUNIT_TSICU
0,55440,195768,228357,89.0,145.0,121.043478,74.0,127.0,106.586957,42.0,...,False,False,True,False,False,False,False,True,False,False
1,76908,126136,221004,63.0,110.0,79.117647,89.0,121.0,106.733333,49.0,...,True,False,False,False,False,False,False,True,False,False
2,95798,136645,296315,81.0,98.0,91.689655,88.0,138.0,112.785714,45.0,...,False,True,False,False,False,False,False,True,False,False
3,40708,102505,245557,76.0,128.0,98.857143,84.0,135.0,106.972973,30.0,...,False,False,False,False,True,False,False,False,True,False
4,28424,127337,225281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,True,False,False,False,False,True


In [23]:
# Save the datasets
csv_path1 = os.path.join("preprocessed_datasets", "06_train_dataset_v3.csv")
df_train3.to_csv(csv_path1, index = False)

csv_path2 = os.path.join("preprocessed_datasets", "07_test_dataset_v3.csv")
df_test3.to_csv(csv_path2, index = False)

csv_path3 = os.path.join("preprocessed_datasets", "08_concat_dataset_v3.csv")
df_concat3.to_csv(csv_path3, index = False)